In [1]:
# from podcast_ad_skipper.Spectrogram_Conversion import create_spectrogram
import pandas as pd
from podcast_ad_skipper.google_cloud import *
import librosa
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery


In [2]:
storage_client = auth_gc_storage()

Authenticated successfully! ✅


In [3]:
bq_client = auth_gc_bigquery()

Authenticated successfully with BigQuery! ✅


In [4]:
prefixes =GCP_PREFIXES[0]
bucket_name = BUCKET_NAME

In [5]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [6]:
open_file = open_gcs_file(file_list[0])

In [7]:
features = get_features_model(file_list)

Processing files: total 5837


In [8]:
features[0][0].shape

(96, 64, 3)

In [44]:
import pandas as pd
from podcast_ad_skipper.main import transform_features_into_dataframe

In [45]:
data = transform_features_into_dataframe(features)
data

,spectrogram,labels,seconds,durations,podcast_names
0,"[[[0.3966400921344757, 0.3966400921344757, 0.3...",0,1000,5864,billionairepersonalitydisorder
1,"[[[0.15291857719421387, 0.15291857719421387, 0...",0,1001,5864,billionairepersonalitydisorder
2,"[[[0.1725386381149292, 0.1725386381149292, 0.1...",0,1002,5864,billionairepersonalitydisorder
3,"[[[0.5269443392753601, 0.5269443392753601, 0.5...",0,1003,5864,billionairepersonalitydisorder


In [46]:
type(data["spectrogram"][0])

str

In [47]:
data['seconds'] = data['seconds'].astype(int)
data['durations'] = data['durations'].astype(int)
data["labels"] = data['labels'].astype(int)

In [48]:
data#['spectrogram'][0]

,spectrogram,labels,seconds,durations,podcast_names
0,"[[[0.3966400921344757, 0.3966400921344757, 0.3...",0,1000,5864,billionairepersonalitydisorder
1,"[[[0.15291857719421387, 0.15291857719421387, 0...",0,1001,5864,billionairepersonalitydisorder
2,"[[[0.1725386381149292, 0.1725386381149292, 0.1...",0,1002,5864,billionairepersonalitydisorder
3,"[[[0.5269443392753601, 0.5269443392753601, 0.5...",0,1003,5864,billionairepersonalitydisorder


In [49]:
data.shape

(4, 5)

In [50]:
# data_to_upload = [
#     {
#         'spectrogram': data["spectrogram"][i],  # Array of floats
#         'labels': data["labels"][i]     # Corresponding integer
#     }
#     for i in range(data.shape[0])
# ]

In [51]:
# data_to_upload[3]["labels"]

In [71]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

In [72]:

append_arrays_to_bq(data, bq_client, table_id)

Authenticated successfully with BigQuery! ✅
LoadJob<project=podcast-ad-skipper, location=EU, id=ffc1b735-ff1b-4369-96c8-557d331cf4bf>
Appended rows to podcast-ad-skipper.Numpy_Arrays_Dataset.Numpy_Arrays_Table


In [73]:
client = bq_client
query = """
SELECT spectrogram from podcast-ad-skipper.Numpy_Arrays_Dataset.test5
"""

    # Run the query
query_job = client.query(query)  # Make an API request.

    # Wait for the job to complete and get the results
results = query_job.result()  # Waits for the job to complete.

results


In [74]:
test = row[0]

In [75]:
np.array(row[0])

array('[[[0.3966400921344757, 0.3966400921344757, 0.3966400921344757], [0.18479086458683014, 0.18479086458683014, 0.18479086458683014], [0.1723078191280365, 0.1723078191280365, 0.1723078191280365], [0.17777226865291595, 0.17777226865291595, 0.17777226865291595], [0.21396349370479584, 0.21396349370479584, 0.21396349370479584], [0.23351538181304932, 0.23351538181304932, 0.23351538181304932], [0.2599582076072693, 0.2599582076072693, 0.2599582076072693], [0.2501581013202667, 0.2501581013202667, 0.2501581013202667], [0.17646966874599457, 0.17646966874599457, 0.17646966874599457], [0.13838638365268707, 0.13838638365268707, 0.13838638365268707], [0.12741288542747498, 0.12741288542747498, 0.12741288542747498], [0.05995243042707443, 0.05995243042707443, 0.05995243042707443], [0.1469297707080841, 0.1469297707080841, 0.1469297707080841], [0.1874151974916458, 0.1874151974916458, 0.1874151974916458], [0.15149132907390594, 0.15149132907390594, 0.15149132907390594], [0.18718501925468445, 0.1871850192

In [76]:
type(test)

str

In [77]:
test.replace('\n,,', ',').replace('\n', '').replace(',,', ',').replace(',]', ']').replace('[,', '[').replace('...', '')

'[[[0.3966400921344757, 0.3966400921344757, 0.3966400921344757], [0.18479086458683014, 0.18479086458683014, 0.18479086458683014], [0.1723078191280365, 0.1723078191280365, 0.1723078191280365], [0.17777226865291595, 0.17777226865291595, 0.17777226865291595], [0.21396349370479584, 0.21396349370479584, 0.21396349370479584], [0.23351538181304932, 0.23351538181304932, 0.23351538181304932], [0.2599582076072693, 0.2599582076072693, 0.2599582076072693], [0.2501581013202667, 0.2501581013202667, 0.2501581013202667], [0.17646966874599457, 0.17646966874599457, 0.17646966874599457], [0.13838638365268707, 0.13838638365268707, 0.13838638365268707], [0.12741288542747498, 0.12741288542747498, 0.12741288542747498], [0.05995243042707443, 0.05995243042707443, 0.05995243042707443], [0.1469297707080841, 0.1469297707080841, 0.1469297707080841], [0.1874151974916458, 0.1874151974916458, 0.1874151974916458], [0.15149132907390594, 0.15149132907390594, 0.15149132907390594], [0.18718501925468445, 0.1871850192546844

In [78]:
test.replace(',]', ']').replace('[,', '[').replace(',,', ',')


'[[[0.3966400921344757, 0.3966400921344757, 0.3966400921344757], [0.18479086458683014, 0.18479086458683014, 0.18479086458683014], [0.1723078191280365, 0.1723078191280365, 0.1723078191280365], [0.17777226865291595, 0.17777226865291595, 0.17777226865291595], [0.21396349370479584, 0.21396349370479584, 0.21396349370479584], [0.23351538181304932, 0.23351538181304932, 0.23351538181304932], [0.2599582076072693, 0.2599582076072693, 0.2599582076072693], [0.2501581013202667, 0.2501581013202667, 0.2501581013202667], [0.17646966874599457, 0.17646966874599457, 0.17646966874599457], [0.13838638365268707, 0.13838638365268707, 0.13838638365268707], [0.12741288542747498, 0.12741288542747498, 0.12741288542747498], [0.05995243042707443, 0.05995243042707443, 0.05995243042707443], [0.1469297707080841, 0.1469297707080841, 0.1469297707080841], [0.1874151974916458, 0.1874151974916458, 0.1874151974916458], [0.15149132907390594, 0.15149132907390594, 0.15149132907390594], [0.18718501925468445, 0.1871850192546844

In [79]:
array = np.array(eval(test))


In [80]:
array.shape

(96, 64, 3)

In [81]:
row_index = 0

for row in results:
    if row_index == 1:  # 1 corresponds to the second row (0-based index)
        spectrogram_value_2nd_row = row[0]
        print(f"Spectrogram value of the 2nd row: {spectrogram_value_2nd_row}")
        break  # Exit the loop once we have the value
    row_index += 1

Spectrogram value of the 2nd row: [[[0.15291857719421387, 0.15291857719421387, 0.15291857719421387], [0.23396970331668854, 0.23396970331668854, 0.23396970331668854], [0.15925797820091248, 0.15925797820091248, 0.15925797820091248], [0.2456597238779068, 0.2456597238779068, 0.2456597238779068], [0.32630741596221924, 0.32630741596221924, 0.32630741596221924], [0.3082353472709656, 0.3082353472709656, 0.3082353472709656], [0.36797186732292175, 0.36797186732292175, 0.36797186732292175], [0.4869249165058136, 0.4869249165058136, 0.4869249165058136], [0.45687389373779297, 0.45687389373779297, 0.45687389373779297], [0.28541749715805054, 0.28541749715805054, 0.28541749715805054], [0.26116520166397095, 0.26116520166397095, 0.26116520166397095], [0.19225771725177765, 0.19225771725177765, 0.19225771725177765], [0.24592244625091553, 0.24592244625091553, 0.24592244625091553], [0.06826477497816086, 0.06826477497816086, 0.06826477497816086], [0.4255426526069641, 0.4255426526069641, 0.4255426526069641], [

In [82]:
spectrogram_value_2nd_row.replace('\n,,', ',').replace('\n', '').replace(',,', ',').replace(',]', ']').replace('[,', '[').replace('...', '')

'[[[0.15291857719421387, 0.15291857719421387, 0.15291857719421387], [0.23396970331668854, 0.23396970331668854, 0.23396970331668854], [0.15925797820091248, 0.15925797820091248, 0.15925797820091248], [0.2456597238779068, 0.2456597238779068, 0.2456597238779068], [0.32630741596221924, 0.32630741596221924, 0.32630741596221924], [0.3082353472709656, 0.3082353472709656, 0.3082353472709656], [0.36797186732292175, 0.36797186732292175, 0.36797186732292175], [0.4869249165058136, 0.4869249165058136, 0.4869249165058136], [0.45687389373779297, 0.45687389373779297, 0.45687389373779297], [0.28541749715805054, 0.28541749715805054, 0.28541749715805054], [0.26116520166397095, 0.26116520166397095, 0.26116520166397095], [0.19225771725177765, 0.19225771725177765, 0.19225771725177765], [0.24592244625091553, 0.24592244625091553, 0.24592244625091553], [0.06826477497816086, 0.06826477497816086, 0.06826477497816086], [0.4255426526069641, 0.4255426526069641, 0.4255426526069641], [0.4029015898704529, 0.40290158987

In [83]:
spectrogram_value_2nd_row.replace(',]', ']').replace('[,', '[').replace(',,', ',')

'[[[0.15291857719421387, 0.15291857719421387, 0.15291857719421387], [0.23396970331668854, 0.23396970331668854, 0.23396970331668854], [0.15925797820091248, 0.15925797820091248, 0.15925797820091248], [0.2456597238779068, 0.2456597238779068, 0.2456597238779068], [0.32630741596221924, 0.32630741596221924, 0.32630741596221924], [0.3082353472709656, 0.3082353472709656, 0.3082353472709656], [0.36797186732292175, 0.36797186732292175, 0.36797186732292175], [0.4869249165058136, 0.4869249165058136, 0.4869249165058136], [0.45687389373779297, 0.45687389373779297, 0.45687389373779297], [0.28541749715805054, 0.28541749715805054, 0.28541749715805054], [0.26116520166397095, 0.26116520166397095, 0.26116520166397095], [0.19225771725177765, 0.19225771725177765, 0.19225771725177765], [0.24592244625091553, 0.24592244625091553, 0.24592244625091553], [0.06826477497816086, 0.06826477497816086, 0.06826477497816086], [0.4255426526069641, 0.4255426526069641, 0.4255426526069641], [0.4029015898704529, 0.40290158987

In [84]:
array2 = np.array(eval(spectrogram_value_2nd_row))

In [85]:
array2.shape

(96, 64, 3)